<h1 style="color:#1f77b4">High-Cost Patient Identification – Master Dataset Preparation (Patient-Level)</h1>

<p>This notebook extracts and engineers a comprehensive patient-level dataset from raw Synthea tables to support modeling and visualization of high-cost healthcare utilization.</p>

<p>The resulting dataset aggregates demographic, clinical, procedural, and organizational features into a single row per patient. It is designed to enable advanced analytics focused on identifying key drivers of disproportionately high healthcare costs at the patient level.</p>

<p>This dataset will serve as the foundation for subsequent phases, including predictive modeling and Tableau-based data storytelling.</p>

<h2 style="color:#9467bd">1. Load the Required Libraries</h2>

<p>Import all necessary Python packages for database connection, data manipulation, and analysis.</p>

In [443]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

<h2 style="color:#9467bd">2. Set-up connection with PostGreSQL database</h2>

<h3 style="color:#8c564b">2.1 Define Database Connection Parameters</h3>

In [447]:
# Database credentials
db_user = 'postgres'
db_pass = 'Sospel2015'
db_host = 'localhost'
db_port = '5432'
db_name = 'readmission'

<h3 style="color:#8c564b">2.2 Create SQLAlchemy Engine</h3>

In [450]:
engine = create_engine(f'postgresql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}')

<h2 style="color:#9467bd">3. Build the Patient-Level Master Dataset</h2>

<h3 style="color:#8c564b">3.1 Load all necessary raw tables as pandas DataFrames</h3>

<p>In this step, we load all relevant raw tables from our PostgreSQL database into pandas DataFrames for processing. These tables form the foundation for our patient-level dataset and contain detailed records on patient demographics, encounters, medications, procedures, diagnoses, care plans, and more.</p>

<p>We ensure that:</p>
<ul>
  <li>All necessary tables are included to support the construction of chronic condition flags, cost decomposition, and preventive care behavior features.</li>
  <li>Date columns are parsed as datetime objects.</li>
  <li>Primary keys (like <code>patient</code> or <code>encounter</code>) are consistently named across DataFrames.</li>
</ul>

<p>This section does not perform any transformations yet — its goal is to prepare the raw inputs for downstream processing in a clean, reproducible way.</p>

In [455]:
# Load all required Synthea tables into pandas DataFrames from PostgreSQL

# Core patient information
patients_df = pd.read_sql('SELECT * FROM patients', engine)

# Healthcare events and service usage
encounters_df = pd.read_sql('SELECT * FROM encounters', engine)
procedures_df = pd.read_sql('SELECT * FROM procedures', engine)
medications_df = pd.read_sql('SELECT * FROM medications', engine)
careplans_df = pd.read_sql('SELECT * FROM careplans', engine)

# Clinical observations and diagnoses
conditions_df = pd.read_sql('SELECT * FROM conditions', engine)
observations_df = pd.read_sql('SELECT * FROM observations_cleaned', engine)  # Already cleaned

# Supplementary context
organizations_df = pd.read_sql('SELECT * FROM organizations', engine)
providers_df = pd.read_sql('SELECT * FROM providers', engine)
allergies_df = pd.read_sql('SELECT * FROM allergies', engine)

# Parse datetime columns for all tables
date_cols = {
    'patients': ['birth_date', 'death_date'],
    'encounters': ['start', 'stop'],
    'procedures': ['date'],
    'medications': ['start', 'stop'],
    'conditions': ['start', 'stop'],
    'careplans': ['start', 'stop'],
    'observations': ['date'],
}

# Apply datetime parsing
for table_name, cols in date_cols.items():
    df = eval(f"{table_name}_df")
    for col in cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

<h3 style="color:#8c564b">3.2 Define Disease Burden Categories</h3>

<p>To assess the clinical complexity of each patient, we categorize them based on the presence of seven core chronic disease groups. Each group is defined by keyword sets drawn from the <code>conditions</code>, <code>medications</code>, and <code>procedures</code> tables. A patient is flagged with a disease category if any keyword matches in any of the relevant fields.</p>

<h2>Chronic Disease Group Definitions</h2>

<table border="1" cellpadding="8" cellspacing="0" style="border-collapse: collapse; font-family: Arial, sans-serif; font-size: 14px;">
  <thead style="background-color: #f2f2f2;">
    <tr>
      <th>Disease Group</th>
      <th>Conditions Keywords</th>
      <th>Medications Keywords</th>
      <th>Procedures Keywords</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td><strong>Diabetes</strong></td>
      <td>diabetes, hyperglycemia, type 1, type 2, mellitus</td>
      <td>insulin, metformin, glipizide, glyburide, pioglitazone, sitagliptin, liraglutide, empagliflozin, canagliflozin, dapagliflozin, acarbose, exenatide, dulaglutide</td>
      <td>foot exam, A1C, retinal screening, glucose monitoring, insulin administration</td>
    </tr>
    <tr>
      <td><strong>Hypertension</strong></td>
      <td>hypertension, high blood pressure, essential hypertension</td>
      <td>lisinopril, amlodipine, atenolol, losartan, beta-blocker, diuretic, hydrochlorothiazide, valsartan, enalapril, clonidine, nifedipine</td>
      <td>BP monitoring, blood pressure check, ambulatory BP monitoring, hypertension screening</td>
    </tr>
    <tr>
      <td><strong>Cardiovascular Disease</strong></td>
      <td>myocardial, angina, heart failure, ischemic, coronary, atrial fibrillation, cardiomyopathy</td>
      <td>statin, atorvastatin, simvastatin, rosuvastatin, aspirin, ACE inhibitor, beta-blocker, nitroglycerin, clopidogrel, amiodarone, warfarin</td>
      <td>ECG, angioplasty, CABG, stent, echocardiogram, cardiac catheterization, stress test, Holter monitor, defibrillator implantation</td>
    </tr>
    <tr>
      <td><strong>Respiratory Disease</strong></td>
      <td>COPD, asthma, emphysema, chronic bronchitis, respiratory failure</td>
      <td>albuterol, bronchodilator, salmeterol, steroid inhaler, fluticasone, budesonide, tiotropium, montelukast, theophylline</td>
      <td>spirometry, pulmonary function test, oxygen therapy, bronchoscopy, nebulization</td>
    </tr>
    <tr>
      <td><strong>Cancer</strong></td>
      <td>carcinoma, cancer, tumor, neoplasm, malignancy, leukemia, lymphoma</td>
      <td>chemotherapy, cisplatin, doxorubicin, tamoxifen, immunotherapy, checkpoint inhibitor, paclitaxel, cyclophosphamide, nivolumab, pembrolizumab</td>
      <td>biopsy, radiation therapy, mastectomy, lumpectomy, chemotherapy infusion, PET scan, bone marrow biopsy</td>
    </tr>
    <tr>
      <td><strong>Mental Health</strong></td>
      <td>depression, anxiety, bipolar, schizophrenia, PTSD, OCD, mental disorder, psychosis</td>
      <td>SSRI, fluoxetine, sertraline, antidepressant, antipsychotic, citalopram, venlafaxine, lithium, risperidone, quetiapine, aripiprazole</td>
      <td>psychotherapy, counseling, psychiatric evaluation, CBT, group therapy, inpatient care</td>
    </tr>
    <tr>
      <td><strong>Kidney Disease</strong></td>
      <td>renal failure, CKD, nephropathy, end stage renal, kidney disease, dialysis</td>
      <td>epoetin, phosphate binder, furosemide, calcitriol, erythropoietin, sevelamer, sodium bicarbonate, cinacalcet</td>
      <td>dialysis, kidney transplant, renal biopsy, arteriovenous fistula creation, peritoneal dialysis</td>
    </tr>
  </tbody>
</table>

In [458]:
# Define disease group keyword sets
disease_keywords = {
    'diabetes': {
        'conditions': ['diabetes', 'hyperglycemia', 'type 1', 'type 2', 'mellitus'],
        'medications': [
            'insulin', 'metformin', 'glipizide', 'glyburide', 'pioglitazone', 'sitagliptin',
            'liraglutide', 'canagliflozin', 'dapagliflozin', 'empagliflozin', 'exenatide',
            'rosiglitazone', 'repaglinide', 'acarbose'
        ],
        'procedures': [
            'foot exam', 'a1c', 'retinal screening', 'eye exam', 'blood glucose test',
            'microalbuminuria test', 'diabetes education'
        ]
    },
    'hypertension': {
        'conditions': ['hypertension', 'high blood pressure', 'essential hypertension'],
        'medications': [
            'lisinopril', 'amlodipine', 'atenolol', 'losartan', 'beta-blocker', 'diuretic',
            'hydrochlorothiazide', 'valsartan', 'enalapril', 'ramipril', 'nifedipine', 'clonidine'
        ],
        'procedures': [
            'bp monitoring', 'blood pressure check', 'ambulatory blood pressure monitoring',
            'renal artery ultrasound'
        ]
    },
    'cardiovascular': {
        'conditions': ['myocardial', 'angina', 'heart failure', 'ischemic', 'coronary', 'atrial fibrillation', 'cardiomyopathy'],
        'medications': [
            'statin', 'atorvastatin', 'simvastatin', 'rosuvastatin', 'aspirin', 'ace inhibitor', 'beta-blocker',
            'nitroglycerin', 'clopidogrel', 'warfarin', 'digoxin', 'amiodarone', 'furosemide', 'carvedilol'
        ],
        'procedures': [
            'ecg', 'angioplasty', 'cabg', 'stent', 'echocardiogram', 'cardiac catheterization',
            'stress test', 'holter monitor', 'pacemaker implantation'
        ]
    },
    'respiratory': {
        'conditions': ['copd', 'asthma', 'emphysema', 'chronic bronchitis', 'respiratory failure'],
        'medications': [
            'albuterol', 'bronchodilator', 'salmeterol', 'steroid inhaler', 'fluticasone', 'budesonide',
            'ipratropium', 'tiotropium', 'theophylline', 'montelukast', 'formoterol'
        ],
        'procedures': [
            'spirometry', 'pulmonary function test', 'oxygen therapy', 'chest x-ray',
            'bronchoscopy', 'nebulizer therapy'
        ]
    },
    'cancer': {
        'conditions': ['carcinoma', 'cancer', 'tumor', 'neoplasm', 'malignancy', 'leukemia', 'lymphoma'],
        'medications': [
            'chemotherapy', 'cisplatin', 'doxorubicin', 'tamoxifen', 'immunotherapy', 'checkpoint inhibitor',
            'trastuzumab', 'paclitaxel', 'bevacizumab', 'letrozole', 'anastrozole'
        ],
        'procedures': [
            'biopsy', 'radiation therapy', 'mastectomy', 'lumpectomy', 'chemotherapy infusion',
            'bone marrow biopsy', 'ct scan', 'mri scan', 'pet scan'
        ]
    },
    'mental_health': {
        'conditions': ['depression', 'anxiety', 'bipolar', 'schizophrenia', 'ptsd', 'ocd', 'mental disorder', 'psychosis'],
        'medications': [
            'ssri', 'fluoxetine', 'sertraline', 'antidepressant', 'antipsychotic', 'citalopram',
            'venlafaxine', 'lithium', 'aripiprazole', 'olanzapine', 'quetiapine', 'risperidone',
            'diazepam', 'lorazepam', 'clonazepam'
        ],
        'procedures': [
            'psychotherapy', 'counseling', 'psychiatric evaluation', 'cbt', 'electroconvulsive therapy',
            'group therapy', 'mental health screening'
        ]
    },
    'kidney_disease': {
        'conditions': ['renal failure', 'ckd', 'nephropathy', 'end stage renal', 'kidney disease', 'dialysis'],
        'medications': [
            'epoetin', 'phosphate binder', 'furosemide', 'calcitriol', 'erythropoietin',
            'sevelamer', 'sodium bicarbonate', 'iron supplement', 'cinacalcet'
        ],
        'procedures': [
            'dialysis', 'kidney transplant', 'renal biopsy', 'peritoneal dialysis',
            'hemodialysis', 'gfr test', 'renal ultrasound'
        ]
    }
}

<h3 style="color:#8c564b">3.3 Disease Tagging at Encounter Level</h3>

<p>In this step, we classify each encounter based on whether the patient had evidence of any of the seven disease categories: 
<strong>diabetes, hypertension, cardiovascular disease, respiratory disease, cancer, mental health,</strong> and 
<strong>kidney disease</strong>.</p>

<p>This tagging is performed using diagnosis <strong>descriptions</strong> from the <code>conditions</code> table, 
procedure <strong>descriptions</strong> from the <code>procedures</code> table, and medication <strong>descriptions</strong> 
from the <code>medications</code> table.</p>

<p>The result is an enriched encounter-level dataset where each encounter is flagged with 0 or more disease categories, 
representing the clinical burden associated with that encounter. This classification will later help guide cost attribution 
based on disease relevance.</p>

<h4 style="color:#1f77b4">3.3.1 Tagging Disease Categories in Encounter-Level Data</h4>

<p>We begin by flagging encounters that exhibit clinical evidence of any disease group. 
This is done by searching for disease-specific keywords within the <code>conditions</code>, 
<code>procedures</code>, and <code>medications</code> tables. To ensure clinical relevance, 
we explicitly identify and exclude preventive or wellness-related encounters — such as routine checkups, 
screenings, or well-child visits — using a boolean <code>is_preventive</code> flag based on 
keywords in the encounter description and encounter class.</p>

<p>Disease tagging is applied only to encounters where <code>is_preventive = False</code>. 
If a disease-related term appears in any of the relevant source tables for a non-preventive encounter, 
we mark that encounter with a binary flag for the corresponding disease group.</p>

<p>This results in one column per disease category in <code>encounters_df</code> and a 
<code>disease_burden_score</code> that reflects the number of disease groups represented 
in each clinically meaningful (non-preventive) encounter.</p>

In [465]:
# --- Step 1: Lowercase for consistent matching ---
conditions_df['description'] = conditions_df['description'].str.lower()
procedures_df['description'] = procedures_df['description'].str.lower()
medications_df['description'] = medications_df['description'].str.lower()
encounters_df['description'] = encounters_df['description'].str.lower()
encounters_df['encounter_class'] = encounters_df['encounter_class'].str.lower()

# --- Step 2: Define preventive-related keywords ---
preventive_keywords = ['wellness', 'check-up', 'checkup', 'well child', 'preventive', 'screening', 'annual physical']
preventive_classes = ['wellness']

# --- Step 3: Identify preventive encounters and add boolean flag ---
encounters_df['is_preventive'] = (
    encounters_df['description'].str.contains('|'.join(preventive_keywords), case=False, na=False) |
    encounters_df['encounter_class'].isin(preventive_classes)
)

# --- Step 3A: Subset valid encounter IDs (non-preventive only) ---
valid_encounter_ids = set(encounters_df.loc[~encounters_df['is_preventive'], 'encounter_id'])

# --- Step 4: Initialize disease columns ---
for disease in disease_keywords:
    encounters_df[disease] = 0

# --- Step 5: Updated flagging function: only tag encounters not marked preventive ---
def flag_disease(df, field, keyword_list, disease_col, id_col='encounter'):
    matches = df[
        df[field].str.contains('|'.join(keyword_list), case=False, na=False) &
        df[id_col].isin(valid_encounter_ids)
    ]
    matched_ids = matches[id_col].unique()
    encounters_df.loc[encounters_df['encounter_id'].isin(matched_ids), disease_col] = 1

# --- Step 6: Apply disease flags (only to non-preventive encounters) ---
for disease, keywords in disease_keywords.items():
    if 'conditions' in keywords:
        flag_disease(conditions_df, 'description', keywords['conditions'], disease)
    if 'medications' in keywords:
        flag_disease(medications_df, 'description', keywords['medications'], disease)
    if 'procedures' in keywords:
        flag_disease(procedures_df, 'description', keywords['procedures'], disease)

# --- Step 7: Compute disease burden score ---
encounters_df['disease_burden_score'] = encounters_df[list(disease_keywords.keys())].sum(axis=1)

In [466]:
encounters_df.head()

,encounter_id,start,stop,patient,organization,provider,payer,encounter_class,code,description,...,reason_description,is_preventive,diabetes,hypertension,cardiovascular,respiratory,cancer,mental_health,kidney_disease,disease_burden_score
0,d0c40d10-8d87-447e-836e-99d26ad52ea5,2010-01-23 17:45:28,2010-01-23 18:10:28,034e9e3b-2def-4559-bb2a-7850888ae060,e002090d-4e92-300e-b41e-7d1f21dee4c6,e6283e46-fd81-3611-9459-0edb1c3da357,6e2f1a2d-27bd-3701-8d08-dae202c58632,ambulatory,185345009,encounter for symptom,...,Acute bronchitis (disorder),False,0,0,0,0,0,0,0,0
1,e88bc3a9-007c-405e-aabc-792a38f4aa2b,2012-01-23 17:45:28,2012-01-23 18:00:28,034e9e3b-2def-4559-bb2a-7850888ae060,772ee193-bb9f-30eb-9939-21e86c8e4da5,6f1d59a7-a5bd-3cf9-9671-5bad2f351c28,6e2f1a2d-27bd-3701-8d08-dae202c58632,wellness,162673000,general examination of patient (procedure),...,None,True,0,0,0,0,0,0,0,0
2,8f104aa7-4ca9-4473-885a-bba2437df588,2001-05-01 15:02:18,2001-05-01 15:17:18,1d604da9-9a81-4ba9-80c2-de3375d59b40,5d4b9df1-93ae-3bc9-b680-03249990e558,af01a385-31d3-3c77-8fdb-2867fe88df2f,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,ambulatory,185345009,encounter for symptom,...,Sinusitis (disorder),False,0,0,0,0,0,0,0,0
3,b85c339a-6076-43ed-b9d0-9cf013dec49d,2011-07-28 15:02:18,2011-07-28 15:17:18,1d604da9-9a81-4ba9-80c2-de3375d59b40,3dc9bb2d-5d66-3e61-bf9a-e234c6433577,bb17e691-262b-3546-93d5-d88e7de93246,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,wellness,162673000,general examination of patient (procedure),...,None,True,0,0,0,0,0,0,0,0
4,dae2b7cb-1316-4b78-954f-fa610a6c6d0e,2010-07-27 12:58:08,2010-07-27 13:28:08,10339b10-3cd1-4ac3-ac13-ec26728cb592,b03dba4f-892f-365c-bfd1-bfcfa7a98d5d,7ed6b84a-b847-3744-9d42-15c42297a0c2,d47b3510-2895-3b70-9897-342d681c769d,wellness,162673000,general examination of patient (procedure),...,None,True,0,0,0,0,0,0,0,0


<h4 style="color:#1f77b4">3.3.2 Tagging Disease Categories in Procedures and Medications Data</h4>

<p>To align with the enriched encounter-level dataset, we replicate disease tagging for the 
<code>procedures</code> and <code>medications</code> tables based on their <code>description</code> fields.</p>

<p>However, to maintain clinical relevance and consistency, tagging is applied <strong>only if the associated 
encounter is non-preventive</strong> — determined using the <code>is_preventive</code> flag in the 
<code>encounters_df</code>. Rows linked to preventive encounters are excluded from disease tagging and 
have their costs assigned to <code>non_critical_procedure_cost</code> or <code>non_critical_medication_cost</code> accordingly.</p>

<p>Each non-preventive row is evaluated against a curated list of disease-specific keywords. If a match is found, 
the row is tagged with a binary flag for that disease, and the full cost of the procedure or 
medication is assigned to the corresponding disease-specific cost column.</p>

<p>If multiple disease matches occur, only the first matched disease (based on a predefined 
priority order) is assigned the cost. This structure ensures that each row contributes fully to only one 
disease category, simplifying cost attribution and preserving interpretability.</p>

<p>During final merging, these rows will be aggregated at the <code>encounter_id</code> level to support precise, 
disease-specific cost analysis at the encounter level while excluding costs from wellness visits.</p>

In [471]:
# Ensure all descriptions are lowercase for consistency
procedures_df['description'] = procedures_df['description'].str.lower()
medications_df['description'] = medications_df['description'].str.lower()

# Step 1: Initialize tagging and cost columns
for df, prefix in [(procedures_df, 'procedure'), (medications_df, 'medication')]:
    for disease in disease_keywords:
        df[disease] = 0
        df[f'{disease}_{prefix}_cost'] = 0.0
    df[f'non_critical_{prefix}_cost'] = 0.0

# Step 2: Create lookup set of non-preventive encounter IDs
non_preventive_ids = set(encounters_df[encounters_df['is_preventive'] == 0]['encounter_id'])

# Step 3: Assign disease tag and cost only to non-preventive encounters
def assign_single_disease(df, desc_col, cost_col, prefix):
    for idx, row in df.iterrows():
        enc_id = row['encounter']
        # Skip if encounter is preventive
        if enc_id not in non_preventive_ids:
            df.at[idx, f'non_critical_{prefix}_cost'] = row[cost_col]
            continue
        
        # Check for disease keyword match
        matched = False
        for disease, keywords in disease_keywords.items():
            source_type = 'procedures' if prefix == 'procedure' else 'medications'
            if source_type in keywords:
                if any(kw in row[desc_col] for kw in keywords[source_type]):
                    df.at[idx, disease] = 1
                    df.at[idx, f'{disease}_{prefix}_cost'] = row[cost_col]
                    matched = True
                    break  # First match wins
        if not matched:
            df.at[idx, f'non_critical_{prefix}_cost'] = row[cost_col]

# Step 4: Apply function
assign_single_disease(procedures_df, 'description', 'base_cost', 'procedure')
assign_single_disease(medications_df, 'description', 'total_cost', 'medication')

In [472]:
procedures_df.head()

,procedure_id,date,patient,encounter,code,description,base_cost,reason_code,reason_description,diabetes,...,cardiovascular_procedure_cost,respiratory,respiratory_procedure_cost,cancer,cancer_procedure_cost,mental_health,mental_health_procedure_cost,kidney_disease,kidney_disease_procedure_cost,non_critical_procedure_cost
0,1,2011-04-30 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,6aa37300-d1b4-48e7-a2f8-5e0f70f48f38,169553002,insertion of subcutaneous contraceptive,14896.56,None,None,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,14896.56
1,2,2010-07-27 12:58:08,10339b10-3cd1-4ac3-ac13-ec26728cb592,dae2b7cb-1316-4b78-954f-fa610a6c6d0e,430193006,medication reconciliation (procedure),726.51,None,None,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,726.51
2,3,2010-11-20 03:04:34,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,7ff86631-0378-4bfc-92ce-1edd697eb18e,430193006,medication reconciliation (procedure),788.50,None,None,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,788.50
3,4,2011-02-07 03:04:34,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,b8f76eba-7795-4dcd-a544-f27ac2ef3d46,117015009,throat culture (procedure),2070.44,195662009,Acute viral pharyngitis (disorder),0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,2070.44
4,5,2011-04-19 03:04:34,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,640837d9-845a-433c-9fad-47426664a69d,117015009,throat culture (procedure),2479.39,195662009,Acute viral pharyngitis (disorder),0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,2479.39


In [473]:
medications_df.head()

,medication_id,start,stop,patient,payer,encounter,code,description,base_cost,payer_coverage,...,cardiovascular_medication_cost,respiratory,respiratory_medication_cost,cancer,cancer_medication_cost,mental_health,mental_health_medication_cost,kidney_disease,kidney_disease_medication_cost,non_critical_medication_cost
0,1,2010-05-05 00:26:23,2011-04-30 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,1e0d6b0e-1711-4a25-99f9-b1c700c9b260,389221,etonogestrel 68 mg drug implant,677.08,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,8124.96
1,2,2011-04-30 00:26:23,2012-04-24 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,6aa37300-d1b4-48e7-a2f8-5e0f70f48f38,389221,etonogestrel 68 mg drug implant,624.09,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,7489.08
2,3,2012-04-24 00:26:23,2013-04-19 00:26:23,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,7253a9f9-6f6d-429a-926a-7b1d424eae3f,748856,yaz 28 day pack,43.32,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,519.84
3,4,2011-05-13 12:58:08,2011-05-27 12:58:08,10339b10-3cd1-4ac3-ac13-ec26728cb592,d47b3510-2895-3b70-9897-342d681c769d,e1ab4933-07a1-49f0-b4bd-05500919061d,313782,acetaminophen 325 mg oral tablet,8.14,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,8.14
4,5,2011-12-08 15:02:18,2011-12-22 15:02:18,1d604da9-9a81-4ba9-80c2-de3375d59b40,b1c428d6-4f07-31e0-90f0-68ffa6ff8c76,792fae81-a007-44b0-8221-46953737b089,562251,amoxicillin 250 mg / clavulanate 125 mg oral t...,11.91,0.0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,11.91


<h3 style="color:#8c564b">3.4 Time-Aware Disease Metrics</h3>

<p>For each disease group with confirmed diagnosis or treatment evidence from a non-preventive encounter, 
we calculate time-aware metrics to enhance clinical severity stratification:</p>
<ul>
  <li><b>disease_duration_years</b>: Integer number of years (rounded down) between the first recorded diagnosis/treatment date and the patient’s last known encounter.</li>
  <li><b>age_at_first_diagnosis</b>: Patient’s age in years (rounded down) at the time of their earliest diagnosis or treatment, calculated from their date of birth.</li>
</ul>
<p>These metrics provide longitudinal insight into chronic disease burden and enable identification of early-onset and long-duration conditions. 
Preventive visits are excluded from diagnosis timelines to maintain clinical relevance.</p>

In [476]:
# Step 0: Get non-preventive encounter IDs
non_preventive_ids = set(encounters_df[encounters_df['is_preventive'] == 0]['encounter_id'])

# Step 1: Filter condition, procedure, and medication tables to only non-preventive encounters
nonprev_conditions_df = conditions_df[conditions_df['encounter'].isin(non_preventive_ids)]
nonprev_procedures_df = procedures_df[procedures_df['encounter'].isin(non_preventive_ids)]
nonprev_medications_df = medications_df[medications_df['encounter'].isin(non_preventive_ids)]

# Step 2: Disease metrics generation
last_encounter_date_df = (
    encounters_df.groupby('patient')['start']
    .max().reset_index()
    .rename(columns={'start': 'last_encounter_date'})
)

disease_metrics = []

for disease, keywords in disease_keywords.items():
    print(f'Processing metrics for: {disease}')

    # First evidence dates from non-preventive encounters
    cond_mask = nonprev_conditions_df['description'].apply(lambda x: any(k in x for k in keywords['conditions']))
    cond_dates = nonprev_conditions_df[cond_mask].groupby('patient')['start'].min().reset_index()
    cond_dates.rename(columns={'start': f'{disease}_first_condition_date'}, inplace=True)

    med_mask = nonprev_medications_df['description'].apply(lambda x: any(k in x for k in keywords['medications']))
    med_dates = nonprev_medications_df[med_mask].groupby('patient')['start'].min().reset_index()
    med_dates.rename(columns={'start': f'{disease}_first_med_date'}, inplace=True)

    proc_mask = nonprev_procedures_df['description'].apply(lambda x: any(k in x for k in keywords['procedures']))
    proc_dates = nonprev_procedures_df[proc_mask].groupby('patient')['date'].min().reset_index()
    proc_dates.rename(columns={'date': f'{disease}_first_proc_date'}, inplace=True)

    # Merge and compute final evidence date
    dates_df = cond_dates.merge(med_dates, on='patient', how='outer')
    dates_df = dates_df.merge(proc_dates, on='patient', how='outer')
    dates_df[f'{disease}_first_evidence_date'] = dates_df[
        [f'{disease}_first_condition_date', f'{disease}_first_med_date', f'{disease}_first_proc_date']
    ].min(axis=1)

    # Merge with patient info
    merged_df = dates_df[['patient', f'{disease}_first_evidence_date']].merge(
        last_encounter_date_df, on='patient', how='left'
    ).merge(
        patients_df[['patient_id', 'birth_date']].rename(columns={'patient_id': 'patient'}),
        on='patient', how='left'
    )

    merged_df[f'{disease}_duration_years'] = np.floor(
        (merged_df['last_encounter_date'] - merged_df[f'{disease}_first_evidence_date'])
        .dt.total_seconds() / (365.25 * 24 * 60 * 60)
    ).astype('Int64')

    merged_df[f'{disease}_age_at_first_diagnosis'] = np.floor(
        (merged_df[f'{disease}_first_evidence_date'] - merged_df['birth_date'])
        .dt.total_seconds() / (365.25 * 24 * 60 * 60)
    ).astype('Int64')

    disease_metrics.append(
        merged_df[['patient', f'{disease}_duration_years', f'{disease}_age_at_first_diagnosis']]
    )

# Final merge
final_disease_metrics_df = disease_metrics.pop(0)
for df in disease_metrics:
    final_disease_metrics_df = final_disease_metrics_df.merge(df, on='patient', how='outer')

# Optional fill (keep <NA> if you want to retain missing logic)
final_disease_metrics_df.fillna(0, inplace=True)

# Preview
final_disease_metrics_df.head()

Processing metrics for: diabetes
Processing metrics for: hypertension
Processing metrics for: cardiovascular
Processing metrics for: respiratory
Processing metrics for: cancer
Processing metrics for: mental_health
Processing metrics for: kidney_disease


,patient,diabetes_duration_years,diabetes_age_at_first_diagnosis,hypertension_duration_years,hypertension_age_at_first_diagnosis,cardiovascular_duration_years,cardiovascular_age_at_first_diagnosis,respiratory_duration_years,respiratory_age_at_first_diagnosis,cancer_duration_years,cancer_age_at_first_diagnosis,mental_health_duration_years,mental_health_age_at_first_diagnosis,kidney_disease_duration_years,kidney_disease_age_at_first_diagnosis
0,01e1f394-7219-4189-bceb-3cbd90cff90b,0,0,0,0,6,46,0,0,0,0,0,0,0,0
1,023a7d29-32b3-4db5-89c8-b88bd7582ec0,0,0,0,0,55,42,0,0,0,0,0,0,0,0
2,0288abb6-633c-40c3-ba0c-66c7d957727e,0,0,0,0,0,0,66,2,0,0,0,0,0,0
3,02f9aadd-72de-4b20-b381-f4c3b1cf7aa3,0,0,21,42,0,0,0,0,1,61,0,0,0,0
4,03172f6e-fb21-4770-8eef-513730174ab7,0,0,0,0,5,1,0,0,0,0,5,1,0,0


<h3 style="color:#8c564b">3.5 Preventive Care Behavior Features</h3>

<p>To capture patients' engagement in preventive healthcare, we analyze wellness-related encounters.
Each encounter in the dataset includes a <code>wellness</code> flag derived from Synthea, which indicates whether it was a 
routine checkup, screening, or other preventive service.</p>

<p>We aggregate this information at the patient level to create simple but clinically meaningful features:</p>
<ul>
  <li><b>preventive_encounters</b>: Total number of wellness-related encounters per patient.</li>
  <li><b>preventive_ratio</b>: Share of all encounters that were preventive in nature.</li>
</ul>

<p>These features provide insight into preventive behavior patterns and may serve as important signals 
in downstream analyses such as readmission risk modeling or patient profiling.</p>

In [479]:
# Preventive encounter count and ratio per patient
preventive_behavior_df = (
    encounters_df.groupby('patient').agg(
        total_encounters=('encounter_id', 'count'),
        preventive_encounters=('is_preventive', 'sum')
    )
    .assign(preventive_ratio=lambda df: df['preventive_encounters'] / df['total_encounters'])
    .reset_index()
)

# Preview output
preventive_behavior_df.head()

,patient,total_encounters,preventive_encounters,preventive_ratio
0,00185faa-2760-4218-9bf5-db301acf8274,22,10,0.454545
1,0042862c-9889-4a2e-b782-fac1e540ecb4,21,16,0.761905
2,0047123f-12e7-486c-82df-53b3a450e365,21,12,0.571429
3,010d4a3a-2316-45ed-ae15-16f01c611674,13,10,0.769231
4,01207ecd-9dff-4754-8887-4652eda231e2,6,6,1.000000


<h3 style="color:#8c564b">3.6 Cumulative Healthcare Utilization Features</h3>

<p>To complement the clinical profile of each patient, we include basic signals of their healthcare utilization
patterns. These features reflect how frequently and broadly the patient has interacted with the healthcare system, 
which may influence care coordination needs and associated costs.</p>

<p>We include:</p>
<ul>
  <li><b>total_encounters</b>: Total number of encounters recorded per patient.</li>
  <li><b>unique_encounter_types</b>: Count of distinct encounter types (e.g., inpatient, outpatient, emergency).</li>
</ul>

In [482]:
# Ensure START and STOP are datetime
encounters_df['start'] = pd.to_datetime(encounters_df['start'])
encounters_df['stop'] = pd.to_datetime(encounters_df['stop'])

# Calculate encounter duration in days
encounters_df['encounter_duration'] = (encounters_df['stop'] - encounters_df['start']).dt.days

# Filter: only keep encounters with duration ≤ 7 days
encounter_df_filtered = encounters_df[encounters_df['encounter_duration'] <= 7].copy()

# Aggregate: basic utilization features
basic_utilization_df = (
    encounter_df_filtered.groupby('patient').agg(
        total_encounters=('encounter_id', 'count'),
        unique_encounter_types=('encounter_class', pd.Series.nunique)
    ).reset_index()
)

# Rename for consistency
basic_utilization_df.rename(columns={'patient': 'patient'}, inplace=True)

# Preview
basic_utilization_df.head()

,patient,total_encounters,unique_encounter_types
0,00185faa-2760-4218-9bf5-db301acf8274,22,3
1,0042862c-9889-4a2e-b782-fac1e540ecb4,21,4
2,0047123f-12e7-486c-82df-53b3a450e365,21,6
3,010d4a3a-2316-45ed-ae15-16f01c611674,13,2
4,01207ecd-9dff-4754-8887-4652eda231e2,6,1


<h3 style="color:#8c564b">3.7 Assemble Final Patient-Level Master DataFrame</h3>

<p>This step constructs the final <strong>patient-level master dataset</strong> by integrating all relevant features engineered from the cleaned and processed raw tables. The dataset combines comprehensive patient insights derived from:</p>

<ul>
  <li><code>encounters_df</code>: Encounter-level clinical and financial details, including disease indicators and readmission risk factors.</li>
  <li><code>procedures_df</code> and <code>medications_df</code>: Procedure- and medication-specific costs linked to chronic conditions and general treatments.</li>
  <li><code>final_disease_metrics_df</code>: Longitudinal metrics capturing duration and age at first diagnosis for major chronic diseases.</li>
  <li><code>preventive_behavior_df</code>: Preventive care engagement quantified by encounter type frequency and ratios.</li>
  <li><code>basic_utilization_df</code>: Overall healthcare utilization patterns measured through encounter counts and encounter type diversity.</li>
</ul>

<p>We also derive and include the following <strong>aggregated cost metrics</strong> per patient:</p>
<ul>
  <li><strong>encounter_cost</strong>: Total <code>total_claim_cost</code> from all encounters</li>
  <li><strong>procedure_cost</strong>: Total <code>base_cost</code> from all procedures</li>
  <li><strong>medication_cost</strong>: Total <code>total_cost</code> from all medications</li>
  <li><strong>total_cost</strong>: Combined total of encounter, procedure, and medication costs</li>
  <li><strong>%_encounter_cost</strong>: Share of encounter cost in total healthcare cost</li>
  <li><strong>%_procedure_cost</strong>: Share of procedure cost in total healthcare cost</li>
  <li><strong>%_medication_cost</strong>: Share of medication cost in total healthcare cost</li>
</ul>

<p>The resulting master dataset enables advanced analyses across clinical conditions, care patterns, resource use, and cost distributions at the patient level, supporting both predictive modeling and dashboard visualization.</p>

<h4 style="color:#1f77b4">3.7.1 Start from patients_df with selected demographics</h4>

In [492]:
# Select demographic features
patient_master_df = patients_df[[
    'patient_id', 'birth_date', 'gender', 'race', 'ethnicity', 'marital_status'
]].copy()
patient_master_df.rename(columns={
    'patient_id': 'patient'
}, inplace=True)

<h4 style="color:#1f77b4">3.7.2 Add age at last encounter</h4>

In [495]:
# Age at last encounter
last_encounter = encounters_df.groupby('patient')['start'].max().reset_index()
last_encounter = last_encounter.merge(patients_df[['patient_id', 'birth_date']], 
                                      left_on='patient', right_on='patient_id', how='left')
last_encounter['age'] = ((last_encounter['start'] - last_encounter['birth_date']).dt.days / 365.25).astype(int)
last_encounter = last_encounter[['patient', 'age']]

# Merge into master
patient_master_df = patient_master_df.merge(last_encounter, on='patient', how='left')

<h4 style="color:#1f77b4">3.7.3 Add total healthcare cost from 3 sources</h4>

In [498]:
# Encounter cost
encounter_cost_df = encounters_df.groupby('patient', as_index=False)['total_claim_cost'] \
                                 .sum().rename(columns={'total_claim_cost': 'encounter_cost'})

# Procedure cost
procedure_cost_df = procedures_df.groupby('patient', as_index=False)['base_cost'] \
                                 .sum().rename(columns={'base_cost': 'procedure_cost'})

# Medication cost
medication_cost_df = medications_df.groupby('patient', as_index=False)['total_cost'] \
                                   .sum().rename(columns={'total_cost': 'medication_cost'})

# Merge all
patient_master_df = patient_master_df.merge(encounter_cost_df, on='patient', how='left')
patient_master_df = patient_master_df.merge(procedure_cost_df, on='patient', how='left')
patient_master_df = patient_master_df.merge(medication_cost_df, on='patient', how='left')

# Fill missing costs with 0
patient_master_df[['encounter_cost', 'procedure_cost', 'medication_cost']] = \
    patient_master_df[['encounter_cost', 'procedure_cost', 'medication_cost']].fillna(0)

# Calculate total cost and cost ratios
patient_master_df['total_cost'] = patient_master_df['encounter_cost'] + \
                                  patient_master_df['procedure_cost'] + \
                                  patient_master_df['medication_cost']

patient_master_df['%_encounter_cost'] = (patient_master_df['encounter_cost'] / patient_master_df['total_cost']).round(4)
patient_master_df['%_procedure_cost'] = (patient_master_df['procedure_cost'] / patient_master_df['total_cost']).round(4)
patient_master_df['%_medication_cost'] = (patient_master_df['medication_cost'] / patient_master_df['total_cost']).round(4)

<h4 style="color:#1f77b4">3.7.4 Merge in additional prepared datasets</h4>

In [501]:
# Disease metrics
patient_master_df = patient_master_df.merge(final_disease_metrics_df, on='patient', how='left')

# Preventive behavior
patient_master_df = patient_master_df.merge(preventive_behavior_df, on='patient', how='left')

# Basic utilization
patient_master_df = patient_master_df.merge(basic_utilization_df, on='patient', how='left')

In [503]:
patient_master_df.head()

,patient,birth_date,gender,race,ethnicity,marital_status,age,encounter_cost,procedure_cost,medication_cost,...,cancer_age_at_first_diagnosis,mental_health_duration_years,mental_health_age_at_first_diagnosis,kidney_disease_duration_years,kidney_disease_age_at_first_diagnosis,total_encounters_x,preventive_encounters,preventive_ratio,total_encounters_y,unique_encounter_types
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,1989-05-25,M,white,hispanic,M,29,774.96,416.69,41.80,...,<NA>,<NA>,<NA>,<NA>,<NA>,6,3,0.500000,6,2
1,034e9e3b-2def-4559-bb2a-7850888ae060,1983-11-14,M,white,nonhispanic,M,34,774.96,1137.47,6.41,...,<NA>,<NA>,<NA>,<NA>,<NA>,6,3,0.500000,6,3
2,10339b10-3cd1-4ac3-ac13-ec26728cb592,1992-06-02,M,white,nonhispanic,M,26,1549.92,20202.46,20.57,...,<NA>,<NA>,<NA>,<NA>,<NA>,12,4,0.333333,12,5
3,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,1978-05-27,F,white,nonhispanic,M,41,2195.72,86631.60,24630.51,...,<NA>,<NA>,<NA>,<NA>,<NA>,17,3,0.176471,17,3
4,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1996-10-18,M,white,nonhispanic,None,22,1679.08,12090.59,6.52,...,<NA>,<NA>,<NA>,<NA>,<NA>,13,7,0.538462,13,3


In [505]:
patient_master_df.drop(columns=['total_encounters_y'], inplace=True)

In [507]:
patient_master_df.rename(columns={'total_encounters_x': 'total_encounters'}, inplace=True)

<h3 style="color:#8c564b">3.8 Clean & Finalize Patient-Level Master Dataset</h3>

<h4 style="color:#1f77b4">3.8.1 Remove Exact Duplicates</h4>

In [511]:
# Check total number of exact duplicates
patient_master_df.duplicated().sum()

0

<p>No duplicates were found</p>

<h4 style="color:#1f77b4">3.8.2 Check Missing Values and Clean Data Types</h4>

<p>
Before finalizing the patient-level master dataset, we identify and address missing values and ensure consistent data types. All columns representing chronic disease duration and age at first diagnosis currently contain <code>NA</code> for patients without the corresponding condition. Since the absence of a diagnosis is meaningful, we impute these missing values with <code>0</code> to preserve interpretability and maintain numeric compatibility during downstream modeling.
</p>

In [516]:
patient_master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1171 entries, 0 to 1170
Data columns (total 32 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   patient                                1171 non-null   object        
 1   birth_date                             1171 non-null   datetime64[ns]
 2   gender                                 1171 non-null   object        
 3   race                                   1171 non-null   object        
 4   ethnicity                              1171 non-null   object        
 5   marital_status                         791 non-null    object        
 6   age                                    1171 non-null   int32         
 7   encounter_cost                         1171 non-null   float64       
 8   procedure_cost                         1171 non-null   float64       
 9   medication_cost                        1171 non-null   float64 

In [518]:
patient_master_df['marital_status'] = patient_master_df['marital_status'].fillna('Missing')

<p>
The following 14 columns are imputed with 0:
</p>

<ul>
  <li><code>diabetes_duration_years</code>, <code>diabetes_age_at_first_diagnosis</code></li>
  <li><code>hypertension_duration_years</code>, <code>hypertension_age_at_first_diagnosis</code></li>
  <li><code>cardiovascular_duration_years</code>, <code>cardiovascular_age_at_first_diagnosis</code></li>
  <li><code>respiratory_duration_years</code>, <code>respiratory_age_at_first_diagnosis</code></li>
  <li><code>cancer_duration_years</code>, <code>cancer_age_at_first_diagnosis</code></li>
  <li><code>mental_health_duration_years</code>, <code>mental_health_age_at_first_diagnosis</code></li>
  <li><code>kidney_disease_duration_years</code>, <code>kidney_disease_age_at_first_diagnosis</code></li>
</ul>

<p>
All other fields are already complete and use consistent data types.</p>

In [521]:
# List of chronic disease-related columns to fill
disease_cols = [
    'diabetes_duration_years', 'diabetes_age_at_first_diagnosis',
    'hypertension_duration_years', 'hypertension_age_at_first_diagnosis',
    'cardiovascular_duration_years', 'cardiovascular_age_at_first_diagnosis',
    'respiratory_duration_years', 'respiratory_age_at_first_diagnosis',
    'cancer_duration_years', 'cancer_age_at_first_diagnosis',
    'mental_health_duration_years', 'mental_health_age_at_first_diagnosis',
    'kidney_disease_duration_years', 'kidney_disease_age_at_first_diagnosis'
]

# Impute missing values with 0
patient_master_df[disease_cols] = patient_master_df[disease_cols].fillna(0)

# Confirm no missing values remain
print(patient_master_df[disease_cols].isnull().sum())

diabetes_duration_years                  0
diabetes_age_at_first_diagnosis          0
hypertension_duration_years              0
hypertension_age_at_first_diagnosis      0
cardiovascular_duration_years            0
cardiovascular_age_at_first_diagnosis    0
respiratory_duration_years               0
respiratory_age_at_first_diagnosis       0
cancer_duration_years                    0
cancer_age_at_first_diagnosis            0
mental_health_duration_years             0
mental_health_age_at_first_diagnosis     0
kidney_disease_duration_years            0
kidney_disease_age_at_first_diagnosis    0
dtype: int64


<h3 style="color:#8c564b">3.9 Export Final patient_master_df</h3>

<p>After completing all data preparation steps—including data cleaning, clinical enrichment, cost aggregation, and feature engineering—we export the finalized <code>patient_master_df</code>. This dataset consolidates patient-level insights by integrating information from multiple Synthea tables: demographics, conditions, costs, encounters, procedures, medications, and preventive care history.</p>

<p>The final DataFrame consists of <strong>clinically and operationally meaningful features</strong> engineered specifically to support the capstone objective: identifying key patient-level factors associated with high healthcare utilization and chronic disease burden.</p>

<p>It includes:</p>
<ul>
  <li>Demographic details such as <code>gender</code>, <code>race</code>, <code>ethnicity</code>, <code>age</code>, and <code>marital_status</code>.</li>
  <li>Cost-related metrics like <code>encounter_cost</code>, <code>procedure_cost</code>, <code>medication_cost</code>, <code>total_cost</code>, and their corresponding percentage contributions.</li>
  <li>Chronic condition indicators, including duration and age at diagnosis for diseases such as <code>diabetes</code>, <code>hypertension</code>, <code>cardiovascular disease</code>, <code>cancer</code>, <code>respiratory disease</code>, <code>kidney disease</code>, and <code>mental health</code>.</li>
  <li>Utilization metrics such as <code>total_encounters</code>, <code>preventive_encounters</code>, <code>preventive_ratio</code>, and <code>unique_encounter_types</code>.</li>
</ul>

<p>Intermediate or redundant columns (e.g., raw timestamps, identifiers, unused flags) have been excluded to ensure clarity, reduce dimensionality, and optimize performance for downstream modeling and Tableau visual analytics.</p>

<p>The final <code>patient_master_df</code> is exported as a CSV file for seamless integration into predictive modeling workflows and interactive data visualization platforms.</p>

In [529]:
patient_master_df.to_csv(r"C:\Users\yvesj\Desktop\Yves\General Assembly\Capstone\Synthea\Dataset for Capstone\patient_master_data.csv", index=False)